# Overview

In this notebook I will train character-level LSTM. The model will train character by character on some text in order to aftterewards produce completely new text also character by character. This example will be based on Anna Karenina text. The goal is to produce a network that will be able to generate a chunk of text based in the same style as Anna Karenina.<br>

The structure of the network.<br>
<img src="images/LSTM4.jpeg" width="500"><br>
Credits: Udacity Computer vision Nanodegree


# Loding and preparing the data for training

In [3]:
# load resources
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [7]:
# open text file and read it as a text
with open("data/anna.txt", "r") as stream:
    text = stream.read()

# show sample
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

Now we need to map that text into integers and to do that following steps were taken:
1. Create a tuple of all distinct characters preent in the text
2. From that tuple create a dictionary where integers are keys
3. Then inverse mapping from relation `int : char` to `char: int`
4. Using dict from point 3. map whole text into a list of corresponding integers

In [9]:
# step 1: use set constructor to get unique chars from whole text. Next make it immutable using tuple
chars = tuple(set(text))
# step 2: create dictionary `int : char`
int2char = dict(enumerate(chars))
# step 3: now inverse the mapping
char2int = {char : integer for integer, char in int2char.items()}
# step 4: map every char in text to the corresponding value in char2int dict, save it as numpy array
encoded = np.array([char2int[char]for char in text])

Let's see again the first line of the text and its encoded version

In [11]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [12]:
encoded[:100]

array([41, 47, 46,  3, 82, 49,  1,  8,  6, 79, 79, 79, 10, 46,  3,  3,  0,
        8, 51, 46, 65, 53, 26, 53, 49, 24,  8, 46,  1, 49,  8, 46, 26, 26,
        8, 46, 26, 53, 55, 49, 72,  8, 49, 78, 49,  1,  0,  8, 21, 12, 47,
       46,  3,  3,  0,  8, 51, 46, 65, 53, 26,  0,  8, 53, 24,  8, 21, 12,
       47, 46,  3,  3,  0,  8, 53, 12,  8, 53, 82, 24,  8, 40, 13, 12, 79,
       13, 46,  0, 67, 79, 79, 16, 78, 49,  1,  0, 82, 47, 53, 12])

Everything is working perfectly :)